This JNotebook is the guide to create an EDA/Pre-processing workflow

In [ ]:
# Loading needed Libaries

#data handling libaries
import numpy as np
import pandas as pd
#plotting libaries
import matplotlib.pyplot as plt
import seaborn as sns
#maschine learning libaries
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import set_config


# Setting random seed 
RSEED = 42

In [ ]:
# Splitting target and features

def split_target_features(df):
    X = df.drop("target", axis=1)
    y = df.target

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, shuffle=True, random_state=RSEED)
    df_train = pd.concat([X_train, y_train], axis=1)
    df_test = pd.concat([X_test, y_test], axis=1)
    print(f'Your data_test shape is {df_test.shape()}')
    print(f'Your data_train shape is {df_train.shape()}')
    return  X_train, X_test, y_train, y_test, df_train, df_test


In [ ]:
def floating_time(df_train):
    df_train["ordinal_date"] = df_train.Date.astype("datetime64").apply(lambda x: x.toordinal())
    return df_train

In [ ]:
def windspeed(df_train):
    df_train["wind_speed"] = df_train.apply(lambda x: np.sqrt(x["u_component_of_wind_10m_above_ground"]**2 + x["v_component_of_wind_10m_above_ground"]**2), axis=1)
    return df_train

In [ ]:
def reduce_df(df_train, df_test):
    cols_short = [col for col in df_train.columns if not "angle" in col \
     and not "sensor" in col \
     and not "ID" in col \
     and not "Date" in col \
     and not "target_" in col \
     and not "water" in col \
     and not "_of_wind" in col \
     and not "CH4" in col]
    return cols_short

In [ ]:
def impute_df(df_train,cols_short):
    set_config(transform_output="pandas")
    imputer = SimpleImputer(strategy="median")
    df_imp = imputer.fit_transform(df_train[cols_short])
    return df_imp, imputer

In [ ]:
def transform_df(df_imp):
    log_cols = ["L3_cloud_optical_depth", "L3_S02_S02_column_number_density_amf", "wind_speed", "L3_O3_effective_temperature", "L3_N02_N02_slant_column_number_density", "L3_O3_O3_column_number_density", "L3_NO2_tropospheric_NO2_column_density", "L3_NO2_NO2_column_number_density", "L3_H2O_H2O_column_number_desnity", "L3_CLOUD_cloud_base_hight", "L3_CLOUD_surface_albedo",
    "L3_HCHO_tropospheric_HCHO_column_number_density_amf", "L3_CO_CO_column_number_density", "specific_humidity_2m_above_ground",
    "L3_CLOUD_cloud_top_hight"]
    df_imp[log_cols] = df_imp[log_cols].applymap(lambda x: np.log(x))
    return df_imp